In [6]:
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from model import Tudui
from torch.utils.tensorboard import SummaryWriter
import time

In [7]:
train_data = torchvision.datasets.CIFAR10(root='./CIFAR', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10(root='./CIFAR', train=False, download=True, transform=torchvision.transforms.ToTensor())

train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000


In [8]:
train_data_loader = DataLoader(train_data, batch_size=64)
test_data_loader = DataLoader(test_data, batch_size=64)

In [9]:
tudui = Tudui()

loss_fn = nn.CrossEntropyLoss()

learning_rate = 1e-2
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)

In [10]:
total_train_step = 0
total_test_step = 0
epoch = 10

writer = SummaryWriter('logs')
start_time = time.time()
for i in range(epoch):
    print("-----第{}轮训练开始-----".format(i+1))

    tudui.train()
    for data in train_data_loader:
        imgs, targets = data
        outputs = tudui(imgs)
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step += 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print("100次训练用时：{}".format(end_time-start_time))
            print("训练次数：{}, Loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar('train_loss', loss.item(), total_train_step)

    total_test_loss = 0
    total_accuracy = 0

    tudui.eval()
    with torch.no_grad():
        for data in test_data_loader:
            imgs, targets = data
            outputs = tudui(imgs)
            loss = loss_fn(outputs, targets)
            total_accuracy += (outputs.argmax(1) == targets).sum().item()
            total_test_loss += loss.item()

    print("第{}轮训练的Loss为：{}".format(i+1, total_test_loss))
    print("第{}轮训练的准确率为：{}".format(i+1, total_accuracy/test_data_size))
    writer.add_scalar('test_loss', total_test_loss, total_test_step)
    writer.add_scalar('accuracy', total_accuracy/test_data_size, total_test_step)
    total_test_step += 1

    torch.save(tudui, './checkpoints/tudui_{}.pth'.format(i+1))
    print("-----第{}轮训练结束，模型已保存-----".format(i+1))

-----第1轮训练开始-----
100次训练用时：1.1438508033752441
训练次数：100, Loss:2.2781498432159424
100次训练用时：2.2810258865356445
训练次数：200, Loss:2.2767744064331055
100次训练用时：3.406118154525757
训练次数：300, Loss:2.2343685626983643
100次训练用时：4.549553394317627
训练次数：400, Loss:2.1427390575408936
100次训练用时：5.686890125274658
训练次数：500, Loss:2.0545363426208496
100次训练用时：6.865301847457886
训练次数：600, Loss:2.0290956497192383
100次训练用时：7.992433786392212
训练次数：700, Loss:1.963754415512085
第1轮训练的Loss为：307.31628680229187
第1轮训练的准确率为：0.3009
-----第1轮训练结束，模型已保存-----
-----第2轮训练开始-----
100次训练用时：10.203277111053467
训练次数：800, Loss:1.7853186130523682
100次训练用时：11.336512804031372
训练次数：900, Loss:1.7722806930541992
100次训练用时：12.471412897109985
训练次数：1000, Loss:1.8712224960327148
100次训练用时：13.661653757095337
训练次数：1100, Loss:1.9485846757888794
100次训练用时：14.803742170333862
训练次数：1200, Loss:1.6517717838287354
100次训练用时：15.94870114326477
训练次数：1300, Loss:1.597155213356018
100次训练用时：17.13761854171753
训练次数：1400, Loss:1.6990834474563599
100次训练用时：18.27797770500183
